In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


Using TensorFlow backend.


In [2]:

GLOVE_DIR = r'E:\GAYANI\intent_recognition\src\workshop\InferSent\GloVe'

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.840B.300d.txt'),'r',encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))



Indexing word vectors.
Found 2195884 word vectors.


In [3]:
BASE_DIR = r'E:\GAYANI\intent_recognition\news20'
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))


Found 19997 texts.


In [4]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [5]:
print('Preparing embedding matrix.')

# prepare embedding matrix  
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
print('num_words:', num_words)

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
print('embedding_matrix:', embedding_matrix.shape)

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        if len(embedding_vector) is 0:
        # doesn't exist, assign a random vector
            embedding_matrix[i] = np.random.randn(EMBEDDING_DIM)
        else:
            embedding_matrix[i] = embedding_vector


# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)



Preparing embedding matrix.
num_words: 20000
embedding_matrix: (20000, 300)



In [6]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 82s 5ms/step - loss: 2.3199 - acc: 0.2315 - val_loss: 1.6235 - val_acc: 0.4254
Epoch 2/10
15998/15998 [==============================] - 81s 5ms/step - loss: 1.2989 - acc: 0.5396 - val_loss: 1.0973 - val_acc: 0.6129
Epoch 3/10
15998/15998 [==============================] - 81s 5ms/step - loss: 0.9344 - acc: 0.6801 - val_loss: 0.9357 - val_acc: 0.6747
Epoch 4/10
15998/15998 [==============================] - 82s 5ms/step - loss: 0.7396 - acc: 0.7438 - val_loss: 0.8374 - val_acc: 0.7337
Epoch 5/10
15998/15998 [==============================] - 81s 5ms/step - loss: 0.5896 - acc: 0.7987 - val_loss: 0.8575 - val_acc: 0.7252
Epoch 6/10
15998/15998 [==============================] - 82s 5ms/step - loss: 0.4676 - acc: 0.8426 - val_loss: 0.8648 - val_acc: 0.7409
Epoch 7/10
15

In [7]:
# # second, prepare text samples and their labels
# print('Processing text dataset')

# texts = ['who are you?','what is you age?','how old are you','year you born', 'what is your name'] #st of text samples
# labels_index = {'name': 0, 'age':1}  # dictionary mapping label name to numeric id
# labels = [0,1,1,1,0]

# print('Found %s texts.' % len(texts))

In [8]:
A = model.predict(x_train)

In [9]:
np.argmax(A)

31

In [10]:
labels_index

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

In [11]:
scores = model.evaluate(x_val, y_val, verbose=1)


3999/3999 [==============================] - 7s 2ms/step


In [12]:
print('Test accuracy:', scores[1])
print('Test loss:', scores[0])

Test accuracy: 0.7816954238857737
Test loss: 0.7731772385543214


In [13]:
ynew = model.predict(x_train)

In [14]:
for i in ynew[0:10]:
    print('predicted val:', np.argmax(i))


predicted val: 11
predicted val: 11
predicted val: 3
predicted val: 17
predicted val: 5
predicted val: 6
predicted val: 11
predicted val: 4
predicted val: 10
predicted val: 18


In [15]:
for i in range(0,10):
    print("true val:", np.argmax(y_train[i]))

true val: 11
true val: 11
true val: 3
true val: 17
true val: 5
true val: 6
true val: 11
true val: 4
true val: 10
true val: 18
